In [20]:
import pandas as pd
import os
import numpy as np
from functools import reduce
import time
import glob
from datetime import datetime
to_datetime = lambda d: pd.to_datetime(d, format='%d/%m/%Y')

In [30]:
dir = '../cotacoes/dados/b3/cotacoes_diarias/'

cotacoes_dataFrame = pd.DataFrame()

for arquivo in os.listdir(dir):
    
    cotacoes = pd.read_csv(dir + arquivo, parse_dates=['date'], date_parser=to_datetime).sort_values(by='date', ascending=True)
    cotacoes['simbolo'] = arquivo[0:-7]
    cotacoes_dataFrame = cotacoes_dataFrame.append(cotacoes)
    

start_date = '1/08/2015'
cot2015_dataFrame = cotacoes_dataFrame[cotacoes_dataFrame['date'] >= to_datetime(start_date)]
cot2015_dataFrame = cot2015_dataFrame.dropna(axis=1, how='any')
cot2015_dataFrame.set_index(['simbolo','date'], inplace=True)
cot2015_dataFrame.head()

open   high    low  close  neg        vol
simbolo date                                                  
ABCB4   2015-08-03  10.30  10.39  10.19  10.26  260   66204800
        2015-08-04  10.26  10.34  10.19  10.29  197   40806200
        2015-08-05  10.49  10.49  10.05  10.10  442   94990300
        2015-08-06  10.10  10.10   9.91   9.95  836  240119800
        2015-08-07  10.10  10.10   9.74   9.90  350  105921400

In [31]:
def chimp_finder(df, mean_size, var_med_thresh):

    # arquivo = 'QUAL3.sa.csv'

    ## Inicio
    inicio = time.perf_counter()
    micos = pd.DataFrame()
    x = y = 0

    ## Varrer os arquivos no diretório


    ## Cálculos
    cotacoes['vol_med'] = cotacoes['vol'].rolling(window=mean_size).mean()      ## Média Móvel do Volume
    cotacoes['var_vol'] = cotacoes['vol']/cotacoes['vol'].shift(1)        ## Variação do Volume no dia
    cotacoes['var_med'] = cotacoes['vol']/cotacoes['vol_med']             ## Variação do Volume em relação a Média Móvel
    cotacoes['saida'] = cotacoes['open'].shift(-1)                        ## Saída da Estratégia
    cotacoes['lucro'] = cotacoes['saida'] - cotacoes['close']             ## Lucro da Estrategia
    cotacoes['var_lucro'] = (cotacoes['lucro']/cotacoes['close'] + 1)   ## Variação % do Lucro
    cotacoes['var_close'] = (cotacoes['saida'] / cotacoes['saida'].shift(1) - 1) * 100 ## Variação % do Fechamento

    ## Seleção
    selecao = (cotacoes['var_vol'] >= 0) & (cotacoes['var_med'] >= var_med_thresh) ## Variação diária ou relativa a média > 10

    if len(micos) == 0:
        micos = cotacoes[selecao].copy()
    else:
        micos = micos.append(cotacoes[selecao])

    ## Contadores
    x += 1
    y += len(cotacoes)

    micos.set_index(['simbolo','date'], inplace=True)
    
    micos = micos.dropna(axis=0, how='any') # removes NaN lines

    ## Fim
    fim = time.perf_counter()
    
    if(len(micos)):
        profit = reduce(lambda x, y: x*y, micos.reset_index()['var_lucro'])
    else:
        profit = 0
    
    return micos, profit

In [27]:
micos, profit = chimp_finder(10, 5)
#micos = (micos[abs(micos['lucro'])<0.25]['lucro']+1)
print('number of chimps: ' + str(len(micos)))
print('acumulated profit: ' + str(profit))
#micos.plot.hist(bins=100, alpha=0.5)
micos.head()


number of chimps: 12012
acumulated profit: 2.615276536904005e+17


open   high    low  close   neg         vol      vol_med  \
simbolo date                                                                    
ABCB4   2011-01-26  14.30  14.31  13.60  13.62  1096  1612634300  263236750.0   
ADHM3   2017-01-17   0.72   0.86   0.70   0.84   409    91251400   17721390.0   
        2017-01-18   0.86   1.15   0.86   1.09  1002   210766600   38271010.0   
        2017-04-05   2.94   3.20   2.76   2.78   414    65402300    9725670.0   
        2017-07-17   2.20   2.60   2.20   2.45   264    56271000    9398620.0   

                      var_vol   var_med  saida  lucro  var_lucro  var_close  
simbolo date                                                                 
ABCB4   2011-01-26  32.435101  6.126175  13.69   0.07   1.005140  -4.265734  
ADHM3   2017-01-17  16.850664  5.149224   0.86   0.02   1.023810  19.444444  
        2017-01-18   2.309736  5.507213   1.23   0.14   1.128440  43.023256  
        2017-04-05  11.431770  6.724709   2.78   0.00   1.000000  -5.442177  
        2017-07-17   2.410771  5.987156   2.55   0.10   1.040816  15.909091

In [28]:
len(micos)

12012

In [32]:
import random
from deap import algorithms, base, creator, tools

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

separator = [[0, 6], [6, 16]] # how to separate genes in sequence

def evaluate(individual):
    dec_individual = sequence_bin2dec(separator, individual)
    
    micos, profit = chimp_finder(cot2015_dataFrame, dec_individual[0], dec_individual[1]/10)
    return (profit,)

toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=16)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

C:\Users\rapha\Anaconda3\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\Users\rapha\Anaconda3\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


In [34]:
# this piece of code translates integers that represent binaries to the related decimal integer
# should be used before calling the parameter evaluation function (strategy) in order to adequte parameters

def sequence_bin2dec(separator, individual):
    sequence = []

    for sep in separator:
        sequence.append(individual[sep[0]:sep[1]])

    dec_sequence = []
    for gene in sequence:
        int_gene = ''
        for idx, el in enumerate(gene):
            int_gene = int_gene + str(el)
        dec_sequence.append(int(int_gene,2))
    
    return(dec_sequence)

In [15]:
separator = [[0, 6], [6, 16]] # how to separate genes in sequence
individual = [random.randint(0, 1) for _ in range(30)]

print(sequence_bin2dec(separator, individual))

[45, 1022]


In [44]:
if __name__ == "__main__":
    pop = toolbox.population(n=2)
    algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=5, verbose=False)
    print(tools.selBest(pop, k=1))
    dec_seq = sequence_bin2dec(separator, tools.selBest(pop, k=1)[0])
    micos, profit = chimp_finder(cot2015_dataFrame, dec_seq[0], dec_seq[1])
    print(profit)
    micos.head()

[[1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1]]
0


In [15]:
best = [1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1]
dec_seq = sequence_bin2dec(separator, best)
#print(dec_seq)
micos, profit = chimp_finder(dec_seq[0], dec_seq[1])

In [39]:
print(profit)
micos.head()

0


,,open,high,low,close,neg,vol,vol_med,var_vol,var_med,saida,lucro,var_lucro,var_close
simbolo,date,,,,,,,,,,,,,
